In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import h5py
import os
dataset_path = r"/content/gdrive/MyDrive/original_data/eccv16_dataset_summe_google_pool5.h5"
dataset = h5py.File(dataset_path, 'r')
key = dataset.keys()
features=[]
gts=[]
vn=[]
for k in key:
        gt  = dataset[k]['gtscore'][...]
        features.append(dataset[k]['features'][...])
        us = dataset[k]['user_summary'][...]
        cp = dataset[k]['change_points'][...]
        nfps = dataset[k]['n_frame_per_seg'][...]
        nf = dataset[k]['n_frames'][...]
        pi = dataset[k]['picks'][...] 
        ns = dataset[k]['n_steps'][...]
        gts.append(dataset[k]['gtsummary'][...])
        vn.append(dataset[k]['video_name'][...])

In [ ]:
len(features)

25

In [ ]:
import numpy as np
def L2Norm(H1,H2):
    distance =0
    for i in range(len(H1)):
        distance += np.square(H1[i]-H2[i])
    return np.sqrt(distance)

In [ ]:
from PIL import Image
from collections import Counter

In [ ]:
matrix_list=[]

In [ ]:
for feature in features:
  sim_matrix = np.zeros([feature.shape[0], feature.shape[0]])
  for i in range(feature.shape[0]):
    flat_array = feature[i]
    RH1 = Counter(flat_array)
    H1 = []
    for k in range(256):
      if k in RH1.keys():
        H1.append(RH1[k])
      else:
        H1.append(0)
    for j in range(feature.shape[0]):
      flat_array2 = feature[j]
      RH2 = Counter(flat_array2)
      H2 = []
      for l in range(256):
        if l in RH2.keys():
          H2.append(RH2[l])
        else:
          H2.append(0)
      sim_matrix[i][j]=L2Norm(H1,H2)
  matrix_list.append(sim_matrix)
  
  

In [ ]:
upper_mat_list=[]
for i in matrix_list:
  upper_mat_list.append(i[np.triu_indices(i.shape[0])])

In [ ]:
len_list=[]
for i in upper_mat_list:
  len_list.append(len(i))
print(max(len_list))
print(min(len_list))
print(len_list)

210925
2080
[45150, 210925, 5886, 2080, 22791, 47586, 83028, 21115, 99681, 11175, 6903, 50086, 33411, 208335, 60031, 43071, 14878, 21736, 24976, 21115, 58996, 57630, 182710, 3741, 55278]


In [ ]:
new_l=[]
for i in upper_mat_list:
  l=list(i)
  l.extend((210925-len(i))*[0])
  new_l.append(l)
  

In [ ]:
gts_l=[]
for i in gts:
  l=list(i)
  l.extend((649-len(i))*[0])
  gts_l.append(l)

In [ ]:
import pandas as pd
X = pd.DataFrame(new_l)
X

In [ ]:
X.to_csv('X_data.csv')

In [ ]:
y = pd.DataFrame(gts_l)
y

In [ ]:
y.to_csv('y_data.csv')

In [ ]:
nx=pd.read_csv("/content/gdrive/MyDrive/X_data (1).csv")
nx.shape

(25, 210926)

In [ ]:
X.shape

(25, 210925)

In [ ]:
from sklearn.datasets import make_multilabel_classification
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

In [ ]:
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(131072, input_dim=n_inputs, activation='relu'))
	model.add(Dense(n_outputs, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

In [ ]:
n_inputs, n_outputs=X.shape[1],y.shape[1]
x_train=X.iloc[:20,:]
y_train=y.iloc[:20,:]
x_test=X.iloc[20:25,:]
y_test=y.iloc[20:25,:]
model = get_model(n_inputs, n_outputs)
model.fit(x_train, y_train, verbose=0, epochs=100)
yhat = model.predict(x_test)
yhat = yhat.round()
acc = accuracy_score(y_test, yhat)
print('>%.3f' % acc)

ResourceExhaustedError: ignored

In [ ]:
print(yhat[3])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.